<a href="https://colab.research.google.com/github/ekaadev/detection_comments/blob/main/detection_positif_and_negatif_comments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install gensim scikit-learn -U
# add library
import gensim
import logging
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

import random
import re
import os
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("leadbest/googlenewsvectorsnegative300")
path2 = kagglehub.dataset_download("atulanandjha/stanford-sentiment-treebank-v2-sst2")

print("Path to dataset files:", path)
print("Path to dataset files:", path2)

100%|██████████| 19.1M/19.1M [00:00<00:00, 119MB/s]

Extracting files...


Path to dataset files: /kaggle/input/googlenewsvectorsnegative300
Path to dataset files: /root/.cache/kagglehub/datasets/atulanandjha/stanford-sentiment-treebank-v2-sst2/versions/30


In [ ]:
# create a models
gmodel = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
def word_extract(word):
  word = word.lower()
  word = re.sub(r'<[^>]+>', ' ', word)
  word = re.sub(r'(\w)\'(\w)', '\1\2', word)
  word = re.sub(r'\W', ' ', word)
  word = re.sub(r'\s+', ' ', word)
  word = word.strip()
  return word.split()

In [ ]:
import csv

# unsupervised training data
unsup_sentences = []

!wget -P /tmp https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xzf /tmp/aclImdb_v1.tar.gz -C /tmp
for dirname in ['/tmp/aclImdb/train/neg', '/tmp/aclImdb/train/pos', '/tmp/aclImdb/test/neg', '/tmp/aclImdb/test/pos']:
  for filename in os.listdir(dirname):
    with open(os.path.join(dirname, filename), 'r') as f:
      words=word_extract(f.read())
      unsup_sentences.append(TaggedDocument(words, [dirname + '/' + filename]))

!wget -P /tmp http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
!tar -xzf /tmp/review_polarity.tar.gz -C /tmp
for dirname in ['/tmp/txt_sentoken/pos', '/tmp/txt_sentoken/neg']:
  for filename in os.listdir(dirname):
    with open(os.path.join(dirname, filename), 'r') as f:
      for i, sended in enumerate(f):
        words = word_extract(sended)
        unsup_sentences.append(TaggedDocument(words, ['%s/%s-%d' % (dirname, filename, i)]))

with open('/root/.cache/kagglehub/datasets/atulanandjha/stanford-sentiment-treebank-v2-sst2/versions/30/SST2-Data/SST2-Data/stanfordSentimentTreebank/stanfordSentimentTreebank/original_rt_snippets.txt', 'r', encoding='utf-8') as f:
    for i, sended in enumerate(f):
        words = word_extract(sended)
        unsup_sentences.append(TaggedDocument(words, [f'rt-{i}']))


--2025-05-07 06:46:16--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘/tmp/aclImdb_v1.tar.gz.7’

aclImdb_v1.tar.gz.7 100%[===================>]  80.23M  17.3MB/s    in 7.7s    

2025-05-07 06:46:25 (10.4 MB/s) - ‘/tmp/aclImdb_v1.tar.gz.7’ saved [84125825/84125825]

URL transformed to HTTPS due to an HSTS policy
--2025-05-07 06:46:54--  https://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.53
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.53|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3127238 (3.0M) [application/x-gzip]
Saving to: ‘/tmp/review_polarity.tar.gz.6’

review_polarity.tar 100%[====

In [ ]:
len(unsup_sentences)

125325

In [ ]:
unsup_sentences[0:1]

[TaggedDocument(words=['fredd', 'dead', 'did', 'the', 'smartest', 'thing', 'it', 'coul', 'e', 'done', 'after', 'the', 'disappointment', 'of', 'the', 'fifth', 'film', 'it', 'started', 'from', 'scratch', 'sure', 'this', 'final', 'film', 'in', 'the', 'saga', 'is', 'silly', 'but', 'at', 'least', 'i', 'original', 'some', 'of', 'the', 'visuals', 'are', 'even', 'a', 'bit', 'breath', 'taking', 'and', 'the', 'story', 'of', 'fredd', 'kid', 'lisa', 'zane', 'returning', 'to', 'town', 'to', 'face', 'her', 'evil', 'father', 'is', 'unique', 'overall', 'the', 'movie', 'is', 'nothing', 'but', 'another', 'cartoon', 'made', 'to', 'get', 'kids', 'in', 'the', 'theater', 'it', 'has', 'a', 'bunch', 'of', 'good', 'actors', 'zane', 'yaphet', 'kotto', 'and', 'lezlie', 'deane', 'who', 'basically', 'look', 'dumb', 'and', 'wander', 'around', 'like', 'sheep', 'ready', 'for', 'slaughter', 'i', 'one', 'sided', 'i', 'a', 'magic', 'trick', 'and', 'in', 'the', 'end', 'i', 'nothing', 'but', 'goofy', 'childish', 'entertai

In [ ]:
class PermuteSentences(object):
  def __init__(self, sentences):
    self.sentences=sentences

  def __iter__(self):
    shuffled = list(self.sentences)
    random.shuffle(shuffled)
    for sentence in shuffled:
      yield sentence

In [ ]:
permuter = PermuteSentences(unsup_sentences)
model = Doc2Vec(permuter, dm=0, hs=1, vector_size=50)

In [ ]:
model.save('/content/reviews.d2v')

In [ ]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip -O /tmp/sentiment_labelled_sentences.zip
!unzip /tmp/sentiment_labelled_sentences.zip -d /content/data

--2025-05-07 07:17:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/00331/sentiment%20labelled%20sentences.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘/tmp/sentiment_labelled_sentences.zip’

/tmp/sentiment_labe     [ <=>                ]  82.21K   462KB/s    in 0.2s    

2025-05-07 07:17:07 (462 KB/s) - ‘/tmp/sentiment_labelled_sentences.zip’ saved [84188]

Archive:  /tmp/sentiment_labelled_sentences.zip
   creating: /content/data/sentiment labelled sentences/
  inflating: /content/data/sentiment labelled sentences/.DS_Store  
   creating: /content/data/__MACOSX/
   creating: /content/data/__MACOSX/sentiment labelled sentences/
  inflating: /content/data/__MACOSX/sentiment labelled sentences/._.DS_Store  
  inflating: /content/data/sentiment labelled sentences/amazon_cells_la

In [ ]:
sentences = []
sentvecs = []
sentiments = []
for filename in ['yelp', 'amazon_cells', 'imdb']:
  # Assuming the files are in a directory called 'data'
  # Replace 'data' with the actual path to your files if different
  file_path = os.path.join('/content/data/sentiment labelled sentences', '%s_labelled.txt' % filename)
  with open(file_path, encoding='utf-8') as f:
    for i, line in enumerate(f):
      line_split = line.strip().split('\t')
      sentences.append(line_split[0])
      # Make sure 'words' is defined and contains the words from the current sentence
      words = word_extract(line_split[0])
      sentvecs.append(model.infer_vector(words, epochs=10))
      sentiments.append(int(line_split[1]))

combined = list(zip(sentences, sentvecs, sentiments))
random.shuffle(combined)
sentences, sentvecs, sentiments = zip(*combined)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=9)
clfrf = RandomForestClassifier()

In [ ]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(clf, sentvecs, sentiments, cv=5)
np.mean(score), np.std(score)

(0.7816666666666666, 0.009249624617007748)

In [ ]:
score = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(score), np.std(score)

(0.7879999999999999, 0.01774510887227489)

In [ ]:
pipeline = make_pipeline(CountVectorizer(), TfidfTransformer(), RandomForestClassifier())
score = cross_val_score(clfrf, sentvecs, sentiments, cv=5)
np.mean(score), np.std(score)

(0.7873333333333334, 0.009463379711052242)